In [1]:
from mpl_toolkits.basemap import Basemap
from multiprocessing import Process,Array,Value
from collections import defaultdict
from datetime import datetime
from scipy.spatial import distance as dst
from datetime import datetime
import matplotlib.pyplot as plt
import shapely.geometry as sp
import numpy as np
import pandas as pd
import pickle as pk
import threading
import itertools
import math
import time
import json

<div>
<font size="4",style="font-family:Ubuntu"> <p> The following cell reads the Flight_Tracks.csv which is the actual paths taken by all flights in the month of August 2013 </p></font><br>
<font size="3">
<b>Input</b> &rarr; Flight_Tracks.csv- (~81 Million Rows) <br>
<b>Output</b> &rarr; Tracks Dataframe </font>
</div>
 

In [1]:
Tracks=pd.read_csv("Flight_Tracks.csv")
Tracks

,id,received,callsign,altitude,ground_speed,latitude_degrees,longitude_degrees,flighthistory_id
0,108923402,2013-09-09 16:27:54+00,AWE404,37000.0,454,39.330002,-91.730003,308536896
1,108923403,2013-09-09 16:27:54+00,EJA955,43000.0,496,39.130001,-95.970001,308759273
2,108923404,2013-09-09 16:27:54+00,JBU223,36000.0,467,37.169998,-98.980003,308510756
3,108923405,2013-09-09 16:27:54+00,UAL436,37000.0,474,38.919998,-97.529999,308546747
4,108923406,2013-09-09 16:28:00+00,SKW6407,33000.0,451,43.169998,-115.269997,308543571
...,...,...,...,...,...,...,...,...
81406058,15,2013-08-21 16:19:15+00,SKW4595,38000.0,444,40.529999,-117.000000,306674139
81406059,20,2013-08-21 16:19:15+00,SWA445,38000.0,421,40.400002,-107.230003,306650840
81406060,22,2013-08-21 16:19:19+00,AWI3772,3000.0,257,40.830002,-73.750000,306663759
81406061,24,2013-08-21 16:19:17+00,SKW4522,8700.0,272,40.799999,-112.120003,306673932


<div>
<font size="4",style="font-family:Ubuntu"> <p> The following cell drops the unnecessary columns and retains the columns needed to conceptualize the concept of average traffic count which is metric 2 </p></font><br>
<font size="3">
<b>Input</b> &rarr; Tracks Dataframe <br>
<b>Output</b> &rarr;realTracks.csv </font>
</div>

In [ ]:
Tracks.sort_values(by=['flighthistory_id'],inplace=True)
Tracks.drop(['id','callsign','altitude','ground_speed'],axis=1,inplace=True)
Tracks.to_csv("realTracks.csv",index=False)

In [4]:
df = pd.read_csv("Outputs/realTracks.csv")
df

,received,latitude_degrees,longitude_degrees,flighthistory_id
0,2013-08-14 16:40:38+00,42.412800,-99.707800,305840319
1,2013-08-14 16:04:08+00,40.530300,-106.111900,305840319
2,2013-08-14 16:03:21+00,40.449200,-106.297500,305840319
3,2013-08-14 16:02:02+00,40.380800,-106.458300,305840319
4,2013-08-14 16:01:14+00,40.291100,-106.663600,305840319
...,...,...,...,...
81406058,2013-09-11 08:54:03+00,42.930000,-71.500000,308963256
81406059,2013-09-11 09:12:37+00,42.419998,-72.129997,308963256
81406060,2013-09-11 09:53:07+00,41.250000,-73.830002,308963256
81406061,2013-09-11 09:17:38+00,42.270000,-72.330002,308963256


In [8]:
flightDates=set(df["received"].str[:10])
flightTimes=set(df["received"].str[:19])

In [9]:
flightDates=list(flightDates)
flightTimes=list(flightTimes)

In [13]:
flightDates=sorted(flightDates)
flightTimes=sorted(list(map(lambda x:datetime.fromisoformat(x),flightTimes)))

In [15]:
flightTimeDict=dict()
for index,val in enumerate(flightTimes):
    flightTimeDict[val]=index
flightTimeDictFile=open("flightTimeDict.pkl","wb")
pk.dump(flightTimeDict,flightTimeDictFile)
flightTimeDictFile.close()

In [18]:
for date in flightDates:
    dftemp=df[df["received"].str[:10]==date]
    dftemp.to_csv(f"{date}.csv",index=False)

In [ ]:
ZoneDict=dict()
ZoneDict[1000000]=[]
ZoneDict[2000000]=[]
ZoneDict[3000000]=[]
ZoneDict[4000000]=[]
ZoneDict[5000000]=[]
ZoneDict[6000000]=[]
ZoneDict[7000000]=[]
CentreOne=dict()
CentreOne[2000000]=[]
ZoneDict[1000000].append(CentreOne)
CentreTwo=dict()
CentreTwo[4000000]=[]
ZoneDict[1000000].append(CentreTwo)
CentreThree=dict()
CentreThree[1500000]=[]
ZoneDict[2000000].append(CentreThree)
CentreFour=dict()
CentreFour[2000000]=[]
ZoneDict[2000000].append(CentreFour)
CentreFive=dict()
CentreFive[2500000]=[]
ZoneDict[2000000].append(CentreFive)
CentreSix=dict()
CentreSix[4000000]=[]
ZoneDict[2000000].append(CentreSix)
CentreSeven=dict()
CentreSeven[1350000]=[]
ZoneDict[3000000].append(CentreSeven)
CentreEight=dict()
CentreEight[1900000]=[]
ZoneDict[3000000].append(CentreEight)
CentreNine=dict()
CentreNine[2400000]=[]
ZoneDict[3000000].append(CentreNine)
CentreTen=dict()
CentreTen[4000000]=[]
ZoneDict[3000000].append(CentreTen)
CentreEleven=dict()
CentreEleven[1000000]=[]
ZoneDict[4000000].append(CentreEleven)
CentreTwelve=dict()
CentreTwelve[1500000]=[]
ZoneDict[4000000].append(CentreTwelve)
CentreThirteen=dict()
CentreThirteen[2000000]=[]
ZoneDict[4000000].append(CentreThirteen)
CentreFourteen=dict()
CentreFourteen[2500000]=[]
ZoneDict[4000000].append(CentreFourteen)
CentreFifteen=dict()
CentreFifteen[4000000]=[]
ZoneDict[4000000].append(CentreFifteen)
CentreSixteen=dict()
CentreSixteen[800000]=[]
ZoneDict[5000000].append(CentreSixteen)
CentreSeventeen=dict()
CentreSeventeen[1250000]=[]
ZoneDict[5000000].append(CentreSeventeen)
CentreEighteen=dict()
CentreEighteen[1700000]=[]
ZoneDict[5000000].append(CentreEighteen)
CentreNineteen=dict()
CentreNineteen[2150000]=[]
ZoneDict[5000000].append(CentreNineteen)
CentreTwenty=dict()
CentreTwenty[2700000]=[]
ZoneDict[5000000].append(CentreTwenty)
CentreTwentyOne=dict()
CentreTwentyOne[4000000]=[]
ZoneDict[5000000].append(CentreTwentyOne)
CentreTwentyTwo=dict()
CentreTwentyTwo[1750000]=[]
ZoneDict[6000000].append(CentreTwentyTwo)
CentreTwentyThree=dict()
CentreTwentyThree[2300000]=[]
ZoneDict[6000000].append(CentreTwentyThree)
CentreTwentyFour=dict()
CentreTwentyFour[4000000]=[]
ZoneDict[6000000].append(CentreTwentyFour)
CentreTwentyFive=dict()
CentreTwentyFive[4000000]=[]
ZoneDict[7000000].append(CentreTwentyFive)
output = pk.load(open("ConvexDict.pkl", "rb"))
CopyDict=copy.deepcopy(ZoneDict)
newDict=defaultdict(dict)
for zone in CopyDict:
    for chunk in CopyDict[zone]:
        inner_key=[key for key in chunk.keys()][0]
        newDict[zone][inner_key]=[]
maxSector=0
for Zone in newDict:
    for Chunk in newDict[Zone]:
        newDict[Zone][Chunk]+=[sectorNumber for sectorNumber in range(maxSector,maxSector+50)]
        maxSector+=50
SectorChunkDict = open("SectorChunkDict.pkl", "wb")
pk.dump(newDict,SectorChunkDict)
SectorChunkDict.close()

In [2]:
m = pk.load(open("M_ConversionMetric.pkl", "rb"))
SectorChunkDict = pk.load(open("SectorChunkDict.pkl", "rb"))
output = pk.load(open("ConvexDict.pkl", "rb"))
flightTimeDict = pk.load(open("flightTimeDict.pkl", "rb"))

In [4]:
def getChunk(Point,SectorChunkDict):
    innerDict=SectorChunkDict[math.ceil(Point[0]/1000000)*1000000]
    Y=0
    for key in innerDict.keys():
        if key >= Point[1]:
            Y=key
            break
    return innerDict[Y] if Y!=0 else []

In [5]:
def findSectorPath(path,sectorPath,z,SectorChunkDict,output):
    sector=-1
    for pointIdx in range(len(path)):
        sectorList=getChunk(path[pointIdx],SectorChunkDict)
        for i in sectorList:
            if len(output[i]) < 3:
                continue
            if sp.Polygon(output[i]).contains(sp.Point(path[pointIdx])):
                sector=i
                break
        if sector!=-1:
            sectorPath.append(sector)
        else:
            z.append(pointIdx)

In [6]:
def getSectorTimings(dfHistory, m, timingDict, flightTimeDict,SectorChunkDict,output):
    dfHistory.sort_values(by=['flighthistory_id','received'],inplace=True)
    dfHistory=dfHistory.reset_index()
    dfHistory.drop(['index'],axis=1,inplace=True)
    east,north=m(dfHistory["longitude_degrees"],dfHistory["latitude_degrees"])
    path=list(zip(east,north))
    sectorPath=[]
    z=[]
    findSectorPath(path,sectorPath,z,SectorChunkDict,output)
    dfHistory.drop(z, axis=0, inplace=True)
    dfHistory=dfHistory.reset_index()
    dfHistory.drop(['index'],axis=1,inplace=True)
    timings=list(map(lambda x:datetime.fromisoformat(x),dfHistory["received"].str[:19]))
    i=0
    while i<len(sectorPath):
        element=sectorPath[i]
        start=i
        while i<len(sectorPath) and sectorPath[i]==element:
            i+=1
        end=i-1
        if end==len(sectorPath):
            end-=1
        timingDict[element].append(sorted([flightTimeDict[timings[start]],flightTimeDict[timings[end]]]))

In [7]:
def getNaiveK(timingDict):
    naiveKDict=dict()
    for sector in timingDict:
        p=timingDict[sector]
        if(len(p)==0):
            naiveKDict[sector]=-1
            continue
        arrival=[x[0] for x in p]
        departure=[x[1] for x in p]
        t = max(departure)
        count = [0] * (t + 2)
        for i in range(len(arrival)):
            count[arrival[i]] += 1
            count[departure[i] + 1] -= 1

        max_event_tm = count[0]

        for i in range(1, t + 1):
            count[i] += count[i - 1]
            if count[max_event_tm] < count[i]:
                max_event_tm = i
        naiveKDict[sector] = count[max_event_tm]
    return naiveKDict

In [5]:
Dates=["2013-08-14","2013-08-15","2013-08-16","2013-08-17","2013-08-18","2013-08-19","2013-08-20","2013-08-21","2013-08-22","2013-08-23"]

In [11]:
for date in Dates:
    DayDf=pd.read_csv(f"Naive K/{date}.csv")
    timingDictMulti={}
    for sector in range(1250):
        timingDictMulti[sector]=[]
    getSectorTimings(DayDf,m,timingDictMulti,flightTimeDict,SectorChunkDict,output)
    timingDictFile=open(f"Naive K/Output/timingDict{date}.pkl","wb")
    pk.dump(timingDictMulti,timingDictFile)
    timingDictFile.close()
    KDict=getNaiveK(timingDictMulti)
    KDictFile=open(f"Naive K/Output/KDict{date}.pkl","wb")
    pk.dump(KDict,KDictFile)
    KDictFile.close()

In [6]:
KDict=[]
for dates in Dates:
    tempDict=pk.load(open(f"Naive K/Output/KDict{dates}.pkl", "rb"))
    tempDict=dict(sorted(tempDict.items(),key=lambda x : x[1], reverse =True))
    KDict.append(tempDict)

In [7]:
import math
KDictFinal=dict()
for sector in range(1250):
    DictSum=0
    for dateDict in KDict:
        DictSum+=dateDict[sector]
    DictAvg=math.ceil(DictSum/10)
    KDictFinal[sector]=DictAvg

In [8]:
KDictFinalFile=open(f"KDict.pkl","wb")
pk.dump(KDictFinal,KDictFinalFile)

In [10]:
KDict = pk.load(open("KDict.pkl", "rb"))
KDict

{0: 15,
 1: 10,
 2: 9,
 3: 8,
 4: 9,
 5: 0,
 6: 7,
 7: 39,
 8: 7,
 9: 8,
 10: 7,
 11: 6,
 12: 6,
 13: 6,
 14: 6,
 15: 4,
 16: 12,
 17: 9,
 18: 8,
 19: 5,
 20: 7,
 21: 5,
 22: 20,
 23: 38,
 24: 9,
 25: 13,
 26: 6,
 27: 5,
 28: 7,
 29: 6,
 30: 6,
 31: 4,
 32: 5,
 33: 4,
 34: 11,
 35: 7,
 36: 6,
 37: 4,
 38: 6,
 39: 3,
 40: 6,
 41: 8,
 42: 14,
 43: 6,
 44: 8,
 45: 10,
 46: 9,
 47: 4,
 48: 9,
 49: 13,
 50: 9,
 51: 7,
 52: 8,
 53: 7,
 54: 12,
 55: 11,
 56: 5,
 57: 4,
 58: 4,
 59: 9,
 60: 7,
 61: 7,
 62: 5,
 63: 6,
 64: 5,
 65: 4,
 66: 6,
 67: 5,
 68: 4,
 69: 5,
 70: 5,
 71: 7,
 72: 4,
 73: 8,
 74: 5,
 75: 4,
 76: 6,
 77: 6,
 78: 6,
 79: 5,
 80: 4,
 81: 4,
 82: 7,
 83: 5,
 84: 2,
 85: 3,
 86: 8,
 87: 4,
 88: 5,
 89: 6,
 90: 6,
 91: 20,
 92: 6,
 93: 7,
 94: 12,
 95: 4,
 96: 4,
 97: 7,
 98: 5,
 99: 7,
 100: 7,
 101: 8,
 102: 6,
 103: 9,
 104: 6,
 105: 6,
 106: 5,
 107: 7,
 108: 5,
 109: 6,
 110: 11,
 111: 7,
 112: 5,
 113: 5,
 114: 7,
 115: 7,
 116: 5,
 117: 5,
 118: 6,
 119: 7,
 120: 3,
 121:

In [5]:
Tracks

,received,latitude_degrees,longitude_degrees,flighthistory_id
0,2013-08-14 16:40:38+00,42.412800,-99.707800,305840319
1,2013-08-14 16:04:08+00,40.530300,-106.111900,305840319
2,2013-08-14 16:03:21+00,40.449200,-106.297500,305840319
3,2013-08-14 16:02:02+00,40.380800,-106.458300,305840319
4,2013-08-14 16:01:14+00,40.291100,-106.663600,305840319
...,...,...,...,...
81406058,2013-09-11 08:54:03+00,42.930000,-71.500000,308963256
81406059,2013-09-11 09:12:37+00,42.419998,-72.129997,308963256
81406060,2013-09-11 09:53:07+00,41.250000,-73.830002,308963256
81406061,2013-09-11 09:17:38+00,42.270000,-72.330002,308963256


In [3]:
TracksWithSpeed=Tracks[['received','ground_speed','flighthistory_id']]
TracksWithSpeed['received']=list(map(lambda x:datetime.fromisoformat(x),f['received'].str[:19]))
TracksWithSpeed.to_csv("realTrackWithSpeed.csv",index=False)

<div>
<font size="4",style="font-family:Ubuntu"> <p> The following cell reads the realTrackWithSpeed.csv which has a mapping between every flight and its corresponding ground speed in knots captured at 1 minute intervals(timestamps). </p></font><br>
<font size="3">
<b>Input</b> &rarr; realTrackWithSpeed.csv- (~81 Million Rows) <br>
<b>Output</b> &rarr; SpeedDict.pkl </font>
</div>

In [2]:
med = pd.read_csv("Outputs/realTrackWithSpeed.csv")
med

,received,ground_speed,flighthistory_id
0,2013-09-09 16:27:54,454,308536896
1,2013-09-09 16:27:54,496,308759273
2,2013-09-09 16:27:54,467,308510756
3,2013-09-09 16:27:54,474,308546747
4,2013-09-09 16:28:00,451,308543571
...,...,...,...
81406058,2013-08-21 16:19:15,444,306674139
81406059,2013-08-21 16:19:15,421,306650840
81406060,2013-08-21 16:19:19,257,306663759
81406061,2013-08-21 16:19:17,272,306673932


In [7]:
med.sort_values('flighthistory_id')
speeds = med.groupby(["flighthistory_id"])["ground_speed"].median()
df_dict = speeds.to_dict()
avg_speed = np.array(list(df_dict.values())).mean()
med_speed = np.median(list(df_dict.values()))
print("The Average Cruise Speed of a flight is ",avg_speed)
print("The Median Cruise Speed of a flight is ",med_speed)
print("The Number of flights whose cruise speed is known",len(df_dict))
speedDictFile=open("Outputs/SpeedDict.pkl","wb")
pk.dump(df_dict,speedDictFile)
speedDictFile.close()

The Average Cruise Speed of a flight is  386.2909277154822
The Median Cruise Speed of a flight is  407.5
The Number of flights whose cruise speed is known 631131


<div>
<font size="4",style="font-family:Ubuntu"> <p> The following cell reads the Flight_History.csv which has the corresponding information about all the flights that took place in the month of August 2013 and the M and ConvexDict pickle files. It outputs a pickle file that has a mapping between all airports in the Flight_History csv file and their corresponding sector within the USA. </p></font><br>
<font size="3">
<b>Input</b> &rarr; Flight_History.csv,M_ConversionMetric.pkl,ConvexDict.pkl,airports.json <br>
<b>Output</b> &rarr; airportSectorDict.pkl </font>
</div>

In [16]:
flights=pd.read_csv("Outputs/Flight_History.csv",low_memory=False)
m=pk.load(open("Outputs/M_ConversionMetric.pkl",'rb'))
ConvexDict=pk.load(open("Outputs/ConvexDict.pkl",'rb'))
data=json.load(open('Inputs/airports.json', encoding = "utf-8"))

cleanFlights=pd.DataFrame()
cleanFlights=flights[['id','departure_airport_icao_code','arrival_airport_icao_code','scheduled_runway_departure','actual_runway_departure','scheduled_runway_arrival','actual_runway_arrival','scheduled_air_time','departure_airport_timezone_offset','arrival_airport_timezone_offset']]
cleanFlights=cleanFlights.dropna()
cleanFlights=cleanFlights.reset_index()
cleanFlights.drop(['index'], axis = 1, inplace = True)
airports=list(set(cleanFlights["departure_airport_icao_code"])) + list(set(cleanFlights["arrival_airport_icao_code"]))
airports=list(set(airports))
airportObjects=dict()
for x in airports:
    if x in data and data[x]["country"]=='US':
        airportObjects[x]=(data[x]["lat"],data[x]["lon"])
for x in airportObjects:
    airportObjects[x]=m(airportObjects[x][1],airportObjects[x][0])
airportToSector=dict()
for airport in airportObjects:
    for i in range(0,1250):
        if len(ConvexDict[i]) < 3:
            continue
        doesContain=sp.Polygon(ConvexDict[i]).contains(sp.Point(airportObjects[airport]))
        if doesContain:
            airportToSector[airport]=i
            break
filehandler = open("Outputs/airportSectorDict.pkl","wb")
pk.dump(airportToSector,filehandler)
filehandler.close()

In [2]:
flights=pd.read_csv("Outputs/Flight_History.csv",low_memory=False)
tracks=pd.read_csv("Outputs/realTracks.csv")
m = pk.load(open("Outputs/M_ConversionMetric.pkl", "rb"))
speeds = pk.load(open("Outputs/SpeedDict.pkl", "rb"))
SectorGraph = pk.load(open("Outputs/SectorGraph.pkl", "rb"))
Centroids = pk.load(open("Outputs/CentroidDict.pkl", "rb"))
ConvexHulls = pk.load(open("Outputs/ConvexDict.pkl", "rb"))
airportSector=pk.load(open("Outputs/airportSectorDict.pkl", "rb"))

In [3]:
flightID=308536851

In [4]:
speedinKnots=speeds[flightID]
flight=flights[flights['id']==flightID]
start=airportSector[flight['departure_airport_icao_code'].values[0]]
end=airportSector[flight['arrival_airport_icao_code'].values[0]]
print(f"Input to GA:\n{start},{end},0,{speedinKnots}")
realDepart=datetime.fromisoformat(flight['actual_runway_departure'].values[0].split('+')[0])
realArrival=datetime.fromisoformat(flight['actual_runway_arrival'].values[0].split('+')[0])
flights[flights['id']==flightID]

Input to GA:
146,7,0,419.0


,id,airline_code,airline_icao_code,flight_number,departure_airport_code,departure_airport_icao_code,arrival_airport_code,arrival_airport_icao_code,published_departure,published_arrival,...,scheduled_runway_arrival,actual_runway_arrival,creator_code,scheduled_air_time,scheduled_block_time,departure_airport_timezone_offset,arrival_airport_timezone_offset,scheduled_aircraft_type,actual_aircraft_type,icao_aircraft_type_actual
696039,308536851,WN,SWA,2957,PHX,KPHX,OAK,KOAK,2013-09-10 04:20:00+00,2013-09-10 06:10:00+00,...,2013-09-10 06:01:00+00,2013-09-10 06:03:00+00,I,93.0,110.0,-7,-7,73W,NaN,B737


In [25]:
paths=[[146,141,102,107,101,152,151,40,47,8,0,7],
       [146,140,185,181,188,154,158,40,47,8,0,7],
       [146,144,118,116,196,189,181,153,154,158,40,47,8,0,7],
       [146,147,131,133,132,24,22,23,46,42,41,6,1,4,7],
       [146,142,111,110,119,191,190,189,181,153,154,158,40,47,8,0,7],
       [146,127,126,143,147,131,133,132,24,22,23,46,42,41,6,1,4,7],
       [146,141,149,109,104,103,43,48,44,9,0,7],
       [146,143,147,131,133,132,21,26,20,29,46,42,41,6,1,4,7]]

In [26]:
%matplotlib qt
fig = plt.figure(pk.load(open("Outputs/Simulator.pkl","rb")))
plt.plot(realX,realY,linewidth=20,c='r',label="Actual Path")
# plt.plot(GAx,GAy,linewidth=20,c='b',label="GA Path")
plt.legend(loc="upper left",prop={'size': 100})
fig.canvas.draw()

In [27]:
singleFlightTrack=tracks[tracks['flighthistory_id']==flightID]
singleFlightTrack=singleFlightTrack.sort_values('received')
lat=singleFlightTrack['latitude_degrees'].values
lon=singleFlightTrack['longitude_degrees'].values
realX,realY=m(lon,lat)
PathLenGA=0
PathLenReal=0
for pt in range(1,len(realX)):
     PathLenReal+=dst.euclidean((realX[pt-1],realY[pt-1]),(realX[pt],realY[pt]))
mpm_speed=(speedinKnots/1.944)*60
i = 0
for pathFromGA in paths:
    GAx=[Centroids[pathFromGA[0]][0]]
    GAy=[Centroids[pathFromGA[0]][1]]

    for Hull in range(len(pathFromGA)-1):
        X=[]
        Y=[]
        Plot=ConvexHulls[pathFromGA[Hull]]
        Plot.append(ConvexHulls[pathFromGA[Hull]][0])
        for pt in Plot:
            X.append(pt[0])
            Y.append(pt[1])
        line1 = sp.LineString(list(Plot))
        for neigh in SectorGraph.neighbors(pathFromGA[Hull]):
            if(neigh==pathFromGA[Hull+1]):
                line2 = sp.LineString([Centroids[pathFromGA[Hull]], Centroids[neigh]])
                ABC=line1.intersection(line2)
                PathLenGA+=dst.euclidean((GAx[-1],GAy[-1]),(ABC.x,ABC.y))
                GAx.append(ABC.x)
                GAy.append(ABC.y)
    PathLenGA+=dst.euclidean((GAx[-1],GAy[-1]),Centroids[pathFromGA[-1]])
    GAx.append(Centroids[pathFromGA[-1]][0])
    GAy.append(Centroids[pathFromGA[-1]][1])
    plt.plot(GAx,GAy,linewidth=20,c='b',label=f"GA Path {i}")
    i+=1
fig.canvas.draw()

In [24]:
%matplotlib qt
fig = plt.figure(pk.load(open("Outputs/Simulator.pkl","rb")))
plt.plot(realX,realY,linewidth=20,c='r',label="Actual Path")
plt.plot(GAx,GAy,linewidth=20,c='b',label="GA Path")
plt.legend(loc="upper left",prop={'size': 100})
fig.canvas.draw()

In [11]:
print(PathLenGA/mpm_speed)
print(PathLenReal/mpm_speed)
print(((realArrival-realDepart).seconds//3600)*60+(((realArrival-realDepart).seconds//60)%60))

99.07421610592469
100.99182707311093
91
